In [33]:
import torch

In [ ]:
tr_gemma2_embeddings = torch.load('tr_gemma2_embeddings.pt')
print(tr_gemma2_embeddings.shape)

In [35]:
# save embbedings split to 6 parts so that we can push to github
for i in range(6):
    torch.save(tr_gemma2_embeddings[i * 5100:(i + 1) * 5100], f'tr_gemma2_embeddings_{i}.pt')


In [37]:
tr_embeddings = []

for i in range(6):
    tr_embeddings.append(torch.load(f'tr_gemma2_embeddings_{i}.pt'))

tr_embeddings = torch.cat(tr_embeddings)

print(tr_embeddings.shape)

/var/folders/gs/crm1zcvj00j1kl0l1lvz4qf40000gn/T/ipykernel_10522/2702517850.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tr_embeddings.append(torch.load(f'tr_gemma2_e

torch.Size([30158, 2304])


In [38]:
from datasets import load_dataset

ds = load_dataset("alibayram/doktorsitesi")
df = ds['train'].to_pandas()
print(df.columns)

Index(['doctor_title', 'doctor_speciality', 'question_content',
       'question_answer'],
      dtype='object')


In [39]:
print(df['doctor_title'].unique())

['Dr. Öğr. Üyesi' 'Uzm. Dr.' 'Op. Dr.' 'Dyt.' 'Yrd. Doç. Dr.' 'Doç. Dr.'
 'Prof. Dr.' 'Dr.' 'Uzm. Kl. Psk.' 'Dt.' 'Dr.Öğr.Üyesi' 'Prof. Dr. Dt.'
 'Veteriner Hekim' 'Uzm. Psk.' 'Psk.' 'Dr. Dt.' 'Vet.' 'Fzt.' 'Uzm. Dyt.'
 'Uzm. Psk. Dan.' 'Psk. Dan.' 'Uzm. Dr. Dt.' 'Psikoterapist'
 'Ergoterapist' 'Doç. Dr. Dt.' 'Uzm. Dt.' 'Çocuk Gelişim Uzmanı'
 'Dr. Psk. Dan.' 'Dr. Öğr. Üyesi Dt.' 'Dil ve Konuşma Terapisti'
 'Dr. Psk.' 'Doç. Dr. Psk. Dan' 'Aile Danışmanı' 'Pedagog'
 'Klinik Psikolog ' 'Dr. Dyt. ']


In [40]:
print(df['doctor_speciality'].unique())

['ortopedi-ve-travmatoloji' 'dermatoloji'
 'ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi'
 'genel-cerrahi-proktoloji'
 'kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji' 'diyetisyen'
 'kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite'
 'fiziksel-tip-ve-rehabilitasyon' 'beyin-ve-sinir-cerrahisi' 'uroloji'
 'kadin-hastaliklari-ve-dogum' 'genel-cerrahi'
 'dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari'
 'dahiliye-ve-ic-hastaliklari-gastroenteroloji' 'goz-hastaliklari'
 'medikal-estetik-sertifikali-tip-doktoru'
 'cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi' 'cocuk-psikiyatrisi'
 'plastik-rekonstruktif-ve-estetik-cerrahi' 'psikoloji'
 'cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari'
 'aile-hekimligi'
 'fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-algoloji'
 'acil-tip' 'cocuk-sagligi-ve-hastaliklari-cocuk-hematoloji'
 'dahiliye-ve-ic-hastaliklari-nefroloji' 'kardiyoloji' 'dis-hekimi'
 'gogus-cerrahis

In [41]:
speciality_mapping = {
    "acil-tip": ["acil-tip"],
    "akupunktur": ["akupunktur", "akupunktur-biorezonans"],
    "algoloji": ["algoloji", "fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-algoloji", "noroloji-noroloji-algoloji"],
    "anatomi": ["anatomi"],
    "anestezi-ve-reanimasyon": ["anestezi-ve-reanimasyon", "anestezi-ve-reanimasyon-algoloji-anestezi-ve-reanimasyon"],
    "aile-hekimligi": ["aile-hekimligi", "pratisyen-hekimlik"],
    "aile-danismani": ["aile-danismani", "psikoloji-aile-danismani-psikolog", "klinik-psikolog-psikoloji-aile-danismani-psikolog", "dil-konusma-bozuklari-b-aile-danismani"],
    "diyetisyen": ["diyetisyen"],
    "beyin-ve-sinir-cerrahisi": ["beyin-ve-sinir-cerrahisi"],
    "cocuk-cerrahisi": ["cocuk-cerrahisi", "cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi", "cocuk-cerrahisi-cocuk-cerrahisi-cocuk-urolojisi-cocuk-urolojisi-uroloji"],
    "cocuk-sagligi-ve-hastaliklari": ["cocuk-sagligi-ve-hastaliklari", "cocuk-sagligi-ve-hastaliklari-cocuk-enfeksiyon-hastaliklari", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-endokrinoloji", "cocuk-sagligi-ve-hastaliklari-cocuk-gastroenterolojisi", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-genetik-hastaliklari", "cocuk-sagligi-ve-hastaliklari-cocuk-hematoloji", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-kardiyolojisi", "cocuk-sagligi-ve-hastaliklari-cocuk-nefrolojisi", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-norolojisi", "cocuk-sagligi-ve-hastaliklari-cocuk-psikiyatrisi", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-romatolojisi", "cocuk-sagligi-ve-hastaliklari-neonatoloji", 
                                      "cocuk-sagligi-ve-hastaliklari-cocuk-gogus-hastaliklari", "cocuk-sagligi-ve-hastaliklari-cocuk-immunolojisi-ve-alerji-hastaliklari"],
    "dermatoloji": ["dermatoloji"],
    "dis-hekimi": ["dis-hekimi", "dis-hekimi-pedodonti", "dis-hekimi-ortodonti", "dis-hekimi-oral-implantoloji", 
                   "dis-hekimi-periodontoloji", "dis-hekimi-restoratif-dis-tedavisi", "dis-hekimi-endodonti", 
                   "dis-hekimi-agiz-dis-ve-cene-cerrahisi", "dis-hekimi-dis-protezi-uzmani", "dis-hekimi-oral-diagnoz-ve-radyoloji",
                   "dis-hekimi-koruyucu-dis-hekimlig"],
    "endokrinoloji-ve-metabolizma-hastaliklari": ["dahiliye-ve-ic-hastaliklari-endokrinoloji-ve-metabolizma-hastaliklari", "endokrinoloji-ve-metabolizma-hastaliklari"],
    "enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji": ["enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji"],
    "fiziksel-tip-ve-rehabilitasyon": ["fiziksel-tip-ve-rehabilitasyon", "fiziksel-tip-ve-rehabilitasyon-fiziksel-tip-ve-rehabilitasyon-romatoloji"],
    "genel-cerrahi": ["genel-cerrahi", "genel-cerrahi-cerrahi-onkoloji", "genel-cerrahi-gastroenteroloji-cerrahisi", "genel-cerrahi-endokrin-cerrahisi", "genel-cerrahi-meme-cerrahisi", "genel-cerrahi-proktoloji"],
    "gogus-cerrahisi": ["gogus-cerrahisi"],
    "gogus-hastaliklari": ["gogus-hastaliklari", "gogus-hastaliklari-alerji-ve-gogus-hastaliklari"],
    "gastroenteroloji": ["dahiliye-ve-ic-hastaliklari-gastroenteroloji", "gastroenteroloji"],
    "histoloji-ve-embriyoloji": ["histoloji-ve-embriyoloji"],
    "immunoloji": ["dahiliye-ve-ic-hastaliklari-immunoloji-ve-alerji-hastaliklari", "dahiliye-ve-ic-hastaliklari-immunoloji"],
    "kadin-hastaliklari-ve-dogum": ["kadin-hastaliklari-ve-dogum", "kadin-hastaliklari-ve-dogum-jinekolojik-onkoloji", 
                                    "kadin-hastaliklari-ve-dogum-ureme-endokrinolojisi-ve-infertilite", "kadin-hastaliklari-ve-dogum-perinatoloji"],
    "kardiyoloji": ["kardiyoloji"],
    "kalp-damar-cerrahisi": ["kalp-damar-cerrahisi", "kalp-damar-cerrahisi-damar-cerrahisi"],
    "kulak-burun-bogaz-hastaliklari": ["kulak-burun-bogaz-hastaliklari"],
    "noroloji": ["noroloji", "noroloji-klinik-norofizyoloji"],
    "nukleer-tip": ["nukleer-tip"],
    "plastik-rekonstruktif-ve-estetik-cerrahi": ["plastik-rekonstruktif-ve-estetik-cerrahi", "ortopedi-ve-travmatoloji-el-cerrahisi-ve-mikrocerrahi",
                                                 "plastik-rekonstruktif-ve-estetik-cerrahi-el-cerrahisi-ve-mikrocerrahi-plastik"],
    "psikiyatri": ["psikiyatri"],
    "psikoloji": ["psikoloji", "psikolojik-danisman", "psikolojik-danisman-psikoloji", "psikoterapi", "psikoloji-psikoonkoloji"],
    "radyoloji": ["radyoloji", "radyoloji-girisimsel-radyoloji", "radyoloji-uroradyoloji"],
    "romatoloji": ["romatoloji-dahiliye-ve-ic-hastaliklari"],
    "spor-hekimligi": ["spor-hekimligi"],
    "tibbi-genetik": ["tibbi-genetik"],
    "tibbi-onkoloji": ["tibbi-onkoloji", "tibbi-onkoloji-dahiliye-ve-ic-hastaliklari"],
    "tibbi-biyokimya": ["tibbi-biyokimya"],
    "tibbi-patoloji": ["tibbi-patoloji"],
    "uroloji": ["uroloji", "uroloji-androloji"],
    "veteriner": ["veteriner"]
}

for key in speciality_mapping:
    df.loc[df['doctor_speciality'].isin(speciality_mapping[key]), 'doctor_speciality'] = key

print(df['doctor_speciality'].unique())

['ortopedi-ve-travmatoloji' 'dermatoloji'
 'plastik-rekonstruktif-ve-estetik-cerrahi' 'genel-cerrahi'
 'kadin-hastaliklari-ve-dogum' 'diyetisyen'
 'fiziksel-tip-ve-rehabilitasyon' 'beyin-ve-sinir-cerrahisi' 'uroloji'
 'endokrinoloji-ve-metabolizma-hastaliklari' 'gastroenteroloji'
 'goz-hastaliklari' 'medikal-estetik-sertifikali-tip-doktoru'
 'cocuk-sagligi-ve-hastaliklari' 'cocuk-psikiyatrisi' 'psikoloji'
 'aile-hekimligi' 'algoloji' 'acil-tip'
 'dahiliye-ve-ic-hastaliklari-nefroloji' 'kardiyoloji' 'dis-hekimi'
 'gogus-cerrahisi' 'tibbi-genetik' 'gogus-hastaliklari'
 'kulak-burun-bogaz-hastaliklari' 'tibbi-farmakoloji'
 'anestezi-ve-reanimasyon' 'veteriner'
 'enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji' 'kalp-damar-cerrahisi'
 'tibbi-biyokimya' 'psikiyatri' 'noroloji' 'geleneksel-ve-tamamlayici-tip'
 'immunoloji' 'romatoloji' 'radyoloji' 'dahiliye-ve-ic-hastaliklari'
 'akupunktur' 'tibbi-patoloji' 'cocuk-cerrahisi' 'tibbi-onkoloji'
 'fizyoterapi' 'radyasyon-onkolojisi' 'spor-hekiml

In [42]:
title_mapping = {
    "dr-ogr-uyesi": ["Dr. Öğr. Üyesi", "Dr.Öğr.Üyesi", "Dr. Öğr. Üyesi Dt."],
    "uzm-dr": ["Uzm. Dr.", "Uzm. Dr. Dt."],
    "op-dr": ["Op. Dr."],
    "dyt": ["Dyt.", "Dr. Dyt.", "Uzm. Dyt."],
    "yrd-doc-dr": ["Yrd. Doç. Dr."],
    "doc-dr": ["Doç. Dr.", "Doç. Dr. Dt.", "Doç. Dr. Psk. Dan"],
    "prof-dr": ["Prof. Dr.", "Prof. Dr. Dt."],
    "dr": ["Dr."],
    "uzm-kl-psk": ["Uzm. Kl. Psk."],
    "dt": ["Dt.", "Dr. Dt.", "Uzm. Dt."],
    "veteriner-hekim": ["Veteriner Hekim", "Vet."],
    "uzm-psk": ["Uzm. Psk.", "Uzm. Psk. Dan."],
    "psk": ["Psk.", "Psikoterapist", "Klinik Psikolog ", "Dr. Psk.", "Dr. Psk. Dan."],
    "psk-dan": ["Psk. Dan.", "Aile Danışmanı"],
    "fzt": ["Fzt."],
    "ergoterapist": ["Ergoterapist"],
    "cocuk-gelisim-uzmani": ["Çocuk Gelişim Uzmanı"],
    "dil-ve-konusma-terapisti": ["Dil ve Konuşma Terapisti"],
    "pedagog": ["Pedagog"]
}

for key in title_mapping:
    df.loc[df['doctor_title'].isin(title_mapping[key]), 'doctor_title'] = key

print(df['doctor_title'].unique())

['dr-ogr-uyesi' 'uzm-dr' 'op-dr' 'dyt' 'yrd-doc-dr' 'doc-dr' 'prof-dr'
 'dr' 'uzm-kl-psk' 'dt' 'veteriner-hekim' 'uzm-psk' 'psk' 'fzt' 'psk-dan'
 'ergoterapist' 'cocuk-gelisim-uzmani' 'dil-ve-konusma-terapisti'
 'pedagog' 'Dr. Dyt. ']


In [43]:
title_mapping = {
    "profesor": ["prof-dr"],
    "docent": ["doc-dr"],
    "dr-ogr-uyesi": ["dr-ogr-uyesi", "yrd-doc-dr"],
    "uzman-doktor": ["uzm-dr", "op-dr"],
    "diyetisyen": ["dyt"],
    "dr-diyetisyen": ["Dr. Dyt."],
    "psikolog": ["psk", "psk-dan"],
    "uzman-psikolog": ["uzm-psk", "uzm-kl-psk"],
    "doktor": ["dr"],
    "dis-hekimi": ["dt"],
    "fizyoterapist": ["fzt"],
    "veteriner": ["veteriner-hekim"],
    "ergoterapist": ["ergoterapist"],
    "cocuk-gelisim-uzmani": ["cocuk-gelisim-uzmani"],
    "dil-ve-konusma-terapisti": ["dil-ve-konusma-terapisti"],
    "pedagog": ["pedagog"]
}

for key in title_mapping:
    df.loc[df['doctor_title'].isin(title_mapping[key]), 'doctor_title'] = key

# Benzersiz unvanları görüntülemek için:
print(df['doctor_title'].unique())

['dr-ogr-uyesi' 'uzman-doktor' 'diyetisyen' 'docent' 'profesor' 'doktor'
 'uzman-psikolog' 'dis-hekimi' 'veteriner' 'psikolog' 'fizyoterapist'
 'ergoterapist' 'cocuk-gelisim-uzmani' 'dil-ve-konusma-terapisti'
 'pedagog' 'Dr. Dyt. ']


In [45]:
specialty_mapping = {
    "cerrahi": [
        "ortopedi-ve-travmatoloji", "genel-cerrahi", "beyin-ve-sinir-cerrahisi",
        "gogus-cerrahisi", "kalp-damar-cerrahisi", "cocuk-cerrahisi", 
        "plastik-rekonstruktif-ve-estetik-cerrahi", "uroloji"
    ],
    "kadın-dogum": [
        "kadin-hastaliklari-ve-dogum", "cocuk-sagligi-ve-hastaliklari", 
        "cocuk-psikiyatrisi", "aile-hekimligi"
    ],
    "dahiliye": [
        "dahiliye-ve-ic-hastaliklari", "dahiliye-ve-ic-hastaliklari-nefroloji",
        "endokrinoloji-ve-metabolizma-hastaliklari", "gastroenteroloji", 
        "immunoloji", "romatoloji", "dahiliye-ve-ic-hastaliklari-geriatri",
        "dahiliye-ve-ic-hastaliklari-hematoloji"
    ],
    "psikiyatri-ve-psikoloji": [
        "psikiyatri", "psikoloji", "klinik-psikolog", "klinik-psikolog-psikoloji",
        "aile-danismani", "pedagoji"
    ],
    "diş-hekimliği": ["dis-hekimi", "pedodonti"],
    "dermatoloji": ["dermatoloji", "medikal-estetik-sertifikali-tip-doktoru"],
    "göz": ["goz-hastaliklari"],
    "radyoloji-ve-onkoloji": ["radyoloji", "radyasyon-onkolojisi", "tibbi-onkoloji", "nukleer-tip"],
    "fizik-tedavi-ve-rehabilitasyon": ["fiziksel-tip-ve-rehabilitasyon", "fizyoterapi", "ergoterapi"],
    "spor-hekimliği": ["spor-hekimligi"],
    "anestezi-ve-algoloji": ["anestezi-ve-reanimasyon", "algoloji"],
    "genetik": ["tibbi-genetik"],
    "farmakoloji": ["tibbi-farmakoloji"],
    "biyokimya": ["tibbi-biyokimya"],
    "patoloji": ["tibbi-patoloji"],
    "enfeksiyon-ve-immunoloji": ["enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji", "immunoloji"],
    "geleneksel-tamamlayici-tip": ["geleneksel-ve-tamamlayici-tip", "akupunktur"],
    "çocuk-gelişimi": ["cocuk-gelisim-uzmani"],
    "konuşma-terapisi": ["dil-konusma-bozuklari-b"],
    "anatomi-ve-histoloji": ["anatomi", "histoloji-ve-embriyoloji"],
    "veterinerlik": ["veteriner"],
    "kardiyoloji": ["kardiyoloji"],
    "göğüs-hastaliklari": ["gogus-hastaliklari"],
    "kulak-burun-bogaz": ["kulak-burun-bogaz-hastaliklari"]
}

for key in speciality_mapping:
    df.loc[df['doctor_speciality'].isin(speciality_mapping[key]), 'doctor_speciality'] = key

print(df['doctor_speciality'].unique())

['ortopedi-ve-travmatoloji' 'dermatoloji'
 'plastik-rekonstruktif-ve-estetik-cerrahi' 'genel-cerrahi'
 'kadin-hastaliklari-ve-dogum' 'diyetisyen'
 'fiziksel-tip-ve-rehabilitasyon' 'beyin-ve-sinir-cerrahisi' 'uroloji'
 'endokrinoloji-ve-metabolizma-hastaliklari' 'gastroenteroloji'
 'goz-hastaliklari' 'medikal-estetik-sertifikali-tip-doktoru'
 'cocuk-sagligi-ve-hastaliklari' 'cocuk-psikiyatrisi' 'psikoloji'
 'aile-hekimligi' 'algoloji' 'acil-tip'
 'dahiliye-ve-ic-hastaliklari-nefroloji' 'kardiyoloji' 'dis-hekimi'
 'gogus-cerrahisi' 'tibbi-genetik' 'gogus-hastaliklari'
 'kulak-burun-bogaz-hastaliklari' 'tibbi-farmakoloji'
 'anestezi-ve-reanimasyon' 'veteriner'
 'enfeksiyon-hastaliklari-ve-klinik-mikrobiyoloji' 'kalp-damar-cerrahisi'
 'tibbi-biyokimya' 'psikiyatri' 'noroloji' 'geleneksel-ve-tamamlayici-tip'
 'immunoloji' 'romatoloji' 'radyoloji' 'dahiliye-ve-ic-hastaliklari'
 'akupunktur' 'tibbi-patoloji' 'cocuk-cerrahisi' 'tibbi-onkoloji'
 'fizyoterapi' 'radyasyon-onkolojisi' 'spor-hekiml